# Challenge

Another approach to identifying fraudulent transactions is to look for outliers in the data. Standard deviation or quartiles are often used to detect outliers. Using this starter notebook, code two Python functions:

* One that uses standard deviation to identify anomalies for any cardholder.

* Another that uses interquartile range to identify anomalies for any cardholder.

## Identifying Outliers using Standard Deviation

In [1]:
# Initial imports
import os
import pandas as pd
import numpy as np
import random
from dotenv import load_dotenv
from sqlalchemy import create_engine

# load postgres URI env variable
load_dotenv()

True

In [2]:
# set postgres database connection from dotenv
postgres_URI = os.getenv("POSTGRES_URI")

# Create a connection to the database
engine = create_engine(postgres_URI)

In [3]:
# query db to join the transaction and credit_card tables
query = """
    SELECT transaction_date, transaction_amount, transaction_card_id, card_holder_id 
    FROM transaction
    JOIN credit_card 
    ON credit_card.card_number_id = transaction_card_id
"""
query_sd = pd.read_sql(query, engine)
query_sd

,transaction_date,transaction_amount,transaction_card_id,card_holder_id
0,2018-01-01 21:35:10,6.22,3561954487988605,13
1,2018-01-01 21:43:12,3.83,5135837688671496,13
2,2018-01-01 22:41:21,9.61,213193946980303,10
3,2018-01-01 23:13:30,19.03,4263694062533017,4
4,2018-01-01 23:15:10,2.95,4498002758300,18
...,...,...,...,...
3495,2018-12-31 01:24:15,4.84,4723783028106084756,15
3496,2018-12-31 03:33:28,3.26,4165305432349489280,10
3497,2018-12-31 05:53:58,10.73,5361779664174555,19
3498,2018-12-31 08:22:17,11.87,4188164051171486,23


In [17]:
# Write function that locates outliers using standard deviation
outliers = []
def locate_outliers(transactions):
    std = np.std(transactions)
    mean = np.mean(transactions)
    # list comprehensions to find outliers: list = [ expression for element in iterable optional condition ]
    # outliers = [x for x in transactions if (x > mean - (2 * std))]
    outliers = [x for x in outliers if not (x < mean + (2 * std))]
    return outliers

In [18]:
# Find anomalous transactions for 3 random card holders
random = np.random.randint(1,25,3)

outliers_one = query_sd.loc[query_sd["card_holder_id"] == random[0], "transaction_amount"]
outliers_two = query_sd.loc[query_sd["card_holder_id"] == random[1], "transaction_amount"]
outliers_three = query_sd.loc[query_sd["card_holder_id"] == random[2], "transaction_amount"]

locate_outliers(outliers_one)

UnboundLocalError: local variable 'outliers' referenced before assignment

In [6]:
locate_outliers(outliers_two)

[1119.0, 1159.0, 1160.0, 1053.0, 1054.0]

In [7]:
locate_outliers(outliers_three)

[1691.0,
 1302.0,
 1789.9999999999998,
 1017.0,
 1056.0,
 1060.0,
 1660.0000000000002,
 1894.0000000000002,
 1033.0]

## Identifying Outliers Using Interquartile Range

In [8]:
# Write a function that locates outliers using interquartile range
outliers = []
def locate_outliers_IQR(data):
    Q1 = np.percentile(data, 25, interpolation = 'midpoint') 
    Q2 = np.percentile(data, 50, interpolation = 'midpoint') 
    Q3 = np.percentile(data, 75, interpolation = 'midpoint')
    IQR = Q3 - Q1 
    low_lim = Q1 - 1.5 * IQR
    up_lim = Q3 + 1.5 * IQR
    for x in data:
        if ((x> up_lim) or (x<low_lim)):
            outliers.append(x)
    return outliers

In [13]:
# Find anomalous transactions for 3 random card holders
random = np.random.randint(1,25,3)

In [14]:
outliers_1 = query_sd.loc[query_sd["card_holder_id"] == random[0], "transaction_amount"]
outliers_2 = np.sort(outliers_1)

outliers_2 = query_sd.loc[query_sd["card_holder_id"] == random[1], "transaction_amount"]
outliers_2 = np.sort(outliers_2)

outliers_3 = query_sd.loc[query_sd["card_holder_id"] == random[2], "transaction_amount"]
outliers_3 = np.sort(outliers_3)

locate_outliers_IQR(outliers_1)

[117.0,
 121.0,
 175.0,
 333.0,
 458.0,
 974.0,
 1077.0,
 1154.0,
 1176.0,
 1769.0000000000002,
 1814.0,
 1839.0,
 389.0,
 1029.0,
 1108.0,
 1145.0,
 1279.0,
 1379.0,
 1398.0,
 1855.9999999999998,
 2001.0000000000002,
 2108.0,
 229.0,
 1131.0,
 1430.0,
 1617.0000000000002,
 1203.0,
 1103.0,
 89.0,
 1803.0,
 178.0,
 393.0,
 1911.0,
 1014.0,
 1634.0]

In [15]:
locate_outliers_IQR(outliers_2)

[117.0,
 121.0,
 175.0,
 333.0,
 458.0,
 974.0,
 1077.0,
 1154.0,
 1176.0,
 1769.0000000000002,
 1814.0,
 1839.0,
 389.0,
 1029.0,
 1108.0,
 1145.0,
 1279.0,
 1379.0,
 1398.0,
 1855.9999999999998,
 2001.0000000000002,
 2108.0,
 229.0,
 1131.0,
 1430.0,
 1617.0000000000002,
 1203.0,
 1103.0,
 89.0,
 1803.0,
 178.0,
 393.0,
 1911.0,
 1014.0,
 1634.0,
 258.0,
 291.0,
 466.0,
 525.0,
 1011.0,
 1035.0,
 1301.0,
 1901.0000000000002]

In [16]:
locate_outliers_IQR(outliers_3)

[117.0,
 121.0,
 175.0,
 333.0,
 458.0,
 974.0,
 1077.0,
 1154.0,
 1176.0,
 1769.0000000000002,
 1814.0,
 1839.0,
 389.0,
 1029.0,
 1108.0,
 1145.0,
 1279.0,
 1379.0,
 1398.0,
 1855.9999999999998,
 2001.0000000000002,
 2108.0,
 229.0,
 1131.0,
 1430.0,
 1617.0000000000002,
 1203.0,
 1103.0,
 89.0,
 1803.0,
 178.0,
 393.0,
 1911.0,
 1014.0,
 1634.0,
 258.0,
 291.0,
 466.0,
 525.0,
 1011.0,
 1035.0,
 1301.0,
 1901.0000000000002]